In [1]:
import pyodbc
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import xml.etree.ElementTree as ET
import pandas as pd
import os
import re
import random

import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing

from sklearn.feature_selection import SelectFromModel, SequentialFeatureSelector
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from scipy import stats
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
 
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import r2_score
from statsmodels.api import OLS, add_constant
from statsmodels.formula.api import mixedlm

<p style="font-size: 16px; color: blue;"> 可用函式 </p>

In [2]:
def strip_all_columns(df):
    df_cleaned = df.copy()
    for col in df_cleaned.columns:
        if pd.api.types.is_string_dtype(df_cleaned[col]) or df_cleaned[col].dtype == object:
            df_cleaned[col] = df_cleaned[col].astype(str).str.strip()
    return df_cleaned

In [3]:
def count_na_and_blank(df):
    result = {}
    for col in df.columns:
        na_count = df[col].isna().sum()
        blank_count = 0
        if df[col].dtype == object or pd.api.types.is_string_dtype(df[col]):
            blank_count = df[col].astype(str).apply(lambda x: x.strip() == '').sum()
        result[col] = {'na': na_count, 'blank_or_space': blank_count}
    return pd.DataFrame(result).T

# summary = count_na_and_blank(df_merge) 
# summary = summary[(summary['na'] > 0) | (summary['blank_or_space'] > 0)]
# print(summary)

In [4]:
# zms2: 分出L1/2 -> L1 L2
def split_layer_rows(df):
    df_base = df.copy()

    mask = (
        (df_base['mtl_type'] == '基板') &
        (df_base['layer_2'].str.contains('/')) &
        (~df_base['layer_2'].str.contains('-')) &
        (~df_base['layer_2'].str.contains('L0'))
    )

    # 不需拆分的資料保留
    df_keep = df_base[~mask].copy()

    # 要拆分的資料
    df_split = df_base[mask].copy()

    # 新資料列清單
    expanded_rows = []

    for _, row in df_split.iterrows():
        try:
            layer_range = row['layer_2'].replace('L', '')  # 去除 'L'
            start, end = map(int, layer_range.split('/'))  # 拆成起始與結束層數
            for i in range(start, end + 1):
                new_row = row.copy()
                new_row['layer_2'] = f'L{i}'
                expanded_rows.append(new_row)
        except:
            continue  # 若轉換失敗則略過

    # 合併原本資料 + 展開資料
    df_result = pd.concat([df_keep, pd.DataFrame(expanded_rows)], ignore_index=True)

    # 可選擇排序（若有需要）
    # df_result = df_result.sort_values(by=['prod_nbr', 'item_nbr', 'layer_2'], ignore_index=True)

    return df_result


In [5]:
def get_layer_scaling(row):
    panel_id = row['Part_nbr']
    layer = row['d_Layer']
    short_col = f"{layer}_short"
    long_col = f"{layer}_long"

    short_val = None
    long_val = None

    if panel_id in df_VAR_indexed.index and short_col in df_VAR.columns and long_col in df_VAR.columns:
        matched_rows = df_VAR.loc[df_VAR['Panel_ID'] == panel_id]

        # 找第一筆非 0 值的 row
        matched_rows = matched_rows[(matched_rows[short_col] != 0) & (matched_rows[long_col] != 0)]

        if not matched_rows.empty:
            matched = matched_rows.iloc[0]
            short_val = matched[short_col]
            long_val = matched[long_col]

    return pd.Series({'short': short_val, 'long': long_val})

In [6]:
def parse_mtl_desc(desc):
    # 去掉廠商名
    desc = desc.split('+長春')[0].split('+南亞')[0].strip()

    # 正則式（最廣泛泛化）
    pattern = r"""
        (?P<thickness>\d+\.\d{2,3})mm          # 玻纖厚度
        [_\-]?
        (?P<cu_cfg>(?:\d+|H)/(?:\d+|H)|H/H)     # 銅箔上下層結構
        (?:-(?P<cu_type>[A-Z0-9\-/]+))?         # 可選銅箔類型
        (?:\+(?P<glass_str>[\dA-Z\*\+\-]+))?    # 可選玻纖編碼（不拆）
    """

    match = re.match(pattern, desc, re.VERBOSE)
    if match:
        cu_cfg = match.group("cu_cfg")
        cu_upper, cu_lower = '', ''
        if '/' in cu_cfg:
            cu_upper, cu_lower = cu_cfg.split('/')
        else:
            cu_upper = cu_lower = None

        def convert_cu(val):
            if val is None:
                return None
            if val.upper() == 'H':
                return 0.5
            try:
                return float(val)
            except:
                return None

        return pd.Series({
            'glass_fabric_thickness(mm)': float(match.group("thickness")),
            'use_cu_upper_thickness': convert_cu(cu_upper),
            'use_cu_lower_thickness': convert_cu(cu_lower),
            'use_cu_type': match.group("cu_type"),
            'glass_fabric_type': match.group("glass_str"),
            'parse_failed_desc': None
        })

    return pd.Series({
        'glass_fabric_thickness(mm)': None,
        'use_cu_upper_thickness': None,
        'use_cu_lower_thickness': None,
        'use_cu_type': None,
        'glass_fabric_type': None,
        'parse_failed_desc': desc
    })


<p style="font-size: 16px; color: blue;"> SQL Server 連接配置 </p>

In [7]:
# # DM
server = 'NTEMES\MES'
username = 'CHIAFU\wilson_liu'
database = 'IM'
trusted_conn = 'yes'

connection_string = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};Trusted_Connection={trusted_conn};'

conn = pyodbc.connect(connection_string)

# 殘銅
server = 'nte403'
username = 'CHIAFU\wilson_liu'
database = 'chiafu'
trusted_conn = 'yes'

connection2_string = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};Trusted_Connection={trusted_conn};'

conn2 = pyodbc.connect(connection2_string)

# 壓合程式
server = 'nte403'
username = 'CHIAFU\wilson_liu'
database = 'wwip'
trusted_conn = 'yes'

connection5_string = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};Trusted_Connection={trusted_conn};'

conn5 = pyodbc.connect(connection5_string)

# recipe
server = 'nte403'
username = 'CHIAFU\wilson_liu'
database = 'wwip'
trusted_conn = 'yes'

connection4_string = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};Trusted_Connection={trusted_conn};'

conn4 = pyodbc.connect(connection4_string)

# bsitemm
server = 'nte403'
username = 'CHIAFU\wilson_liu'
database = 'erp'
trusted_conn = 'yes'

connection3_string = f'DRIVER={{SQL Server}};SERVER={server};DATABASE={database};UID={username};Trusted_Connection={trusted_conn};'

conn3 = pyodbc.connect(connection3_string)

<p style="font-size: 16px; color: blue;"> DM </p>

In [8]:
# 查詢資料 JOIN (用AVGX 和 AVGY)
query_join_IM = '''
SELECT 
    m.idx, 
    m.Part_nbr, 
    m.Lot_nbr, 
    m.Layer, 
    m.CamX, 
    m.CamY,
    m.Credate,
    d.Layer AS d_Layer,
    d.PSVariationX AS d_PSVarX, 
    d.PSVariationY AS d_PSVarY,
    d.AVGX,
    d.AVGY,
    s.kind
FROM 
    DRY_XrayProduction_M m
    
LEFT JOIN 
    DRY_XrayProduction_D d ON m.idx = d.idx

LEFT JOIN 
    nte403.chiafu.dbo.slss s ON m.Part_nbr = s.part_nbr
WHERE 
    m.Credate > '2024-01-01' 
    AND NOT (m.Credate >= '2025-01-01' AND m.Credate <= '2025-05-26')
'''
# m.Layer = d.Layer 不會對應到 811S002A4GN

In [9]:
df_DM = pd.read_sql(query_join_IM, conn)
df_DM = strip_all_columns(df_DM)
df_DM = df_DM.drop(columns=['idx'])
# df_DM = df_DM[df_DM['kind'] != 'N']
df_DM = df_DM[df_DM['kind'] == 'I']
df_DM = df_DM.drop(columns=['kind'])
df_DM = df_DM.drop_duplicates()

C:\Users\wilson_liu\AppData\Local\Temp\ipykernel_9360\1173620096.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_DM = pd.read_sql(query_join_IM, conn)


In [10]:
# 確保 Credate 是 datetime 格式
df_DM['Credate'] = pd.to_datetime(df_DM['Credate'])

# 找出每個 group 裡 Credate 最大的那一筆資料的 index
idx = df_DM.groupby(['Part_nbr', 'Lot_nbr', 'Layer', 'd_Layer'])['Credate'].idxmax()

# 選出這些資料
df_DM = df_DM.loc[idx].reset_index(drop=True)

<p style="font-size: 16px; color: blue;">殘銅</p>

In [11]:
# 查詢資料 JOIN 殘銅
# 串入M 檔的料號，先串好，再挑時間

query_join_zms = '''
SELECT
    a1.prod_nbr, a1.layer AS layer_full, a1.iss_x, a1.iss_y, a1.creat_date,
    s6.it3, s6.it5, s6.layer
FROM 
    zms1a a1
INNER JOIN 
    zms6 s6
    ON a1.prod_nbr = s6.prod_nbr AND a1.layer = s6.layer    
WHERE 
    s6.cused='51' 
    AND a1.creat_date >= '2024-01-01' AND NOT ( a1.creat_date >= '2025-01-01' AND a1.creat_date <= '2025-05-26') 
'''
df_zms = pd.read_sql(query_join_zms, conn2)
df_zms =df_zms.sort_values(by='creat_date', ascending=False)
df_zms = strip_all_columns(df_zms)

C:\Users\wilson_liu\AppData\Local\Temp\ipykernel_9360\2986971347.py:17: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_zms = pd.read_sql(query_join_zms, conn2)


In [12]:
# 'prod_nbr', 'layer_full' 若相同，留最新
# 確保 creat_date 是 datetime 格式
df_zms['creat_date'] = pd.to_datetime(df_zms['creat_date'])

# 找出每個 group 裡 Credate 最大的那一筆資料的 index
idx = df_zms.groupby(['prod_nbr', 'layer_full'])['creat_date'].idxmax()

# 選出這些資料
df_zms = df_zms.loc[idx].reset_index(drop=True)
df_zms = df_zms.drop_duplicates()

In [13]:
# 層重複交疊問題
# df_zms[df_zms['prod_nbr']=='395A024A33H']

In [14]:
# 轉回數字格式
df_zms['it3'] = pd.to_numeric(df_zms['it3'], errors='coerce')
df_zms['it5'] = pd.to_numeric(df_zms['it5'], errors='coerce')
df_zms['iss_x'] = pd.to_numeric(df_zms['iss_x'], errors='coerce')
df_zms['iss_y'] = pd.to_numeric(df_zms['iss_y'], errors='coerce')

#  C面 / S面 殘銅率
denominator = (df_zms['iss_x'] * df_zms['iss_y'] / 92903.04) + 0.001
df_zms['C-Residual Copper Rate'] = df_zms['it3'] / denominator
df_zms['S-Residual Copper Rate'] = df_zms['it5'] / denominator

df_zms = df_zms[['prod_nbr', 'layer', 'C-Residual Copper Rate', 'S-Residual Copper Rate']]

# df_zms.shape

In [15]:
df_residual_expanded = []

for _, row in df_zms.iterrows():
    layer_raw = str(row['layer']).replace(" ", "").upper()  # 確保一致性
    raw_parts = layer_raw.split("/")  # e.g. ['L6', '7']

    # 安全檢查：忽略非2層格式的資料（例如 1 層或超過2層）
    if len(raw_parts) != 2:
        continue

    def normalize(layer_str):
        m = re.search(r'\d+', layer_str)
        return f'L{m.group()}' if m else None

    layer1 = normalize(raw_parts[0])
    layer2 = normalize(raw_parts[1])
    
    if layer1 is None or layer2 is None:
        continue  # 忽略格式錯誤的

    df_residual_expanded.append({
        'prod_nbr': row['prod_nbr'],
        'd_Layer': layer1,
        'Residual Copper Rate': row['C-Residual Copper Rate']
    })
    df_residual_expanded.append({
        'prod_nbr': row['prod_nbr'],
        'd_Layer': layer2,
        'Residual Copper Rate': row['S-Residual Copper Rate']
    })

# 建立新的展平後的殘銅率表
df_residual_flat = pd.DataFrame(df_residual_expanded)
df_residual_flat = df_residual_flat.drop_duplicates()
df_residual_flat = df_residual_flat.groupby(['prod_nbr', 'd_Layer'])['Residual Copper Rate'].mean().reset_index() # 取平均，處理層重複交疊問題

<p style="font-size: 16px; color: blue;">壓合程式、壓合recipe</p>

In [16]:
# df_cpinm( part_nbr、lot_nbr、layer) and credate -> df_DM_IM_zms ('Part_nbr', 'Lot_nbr', 'Layer')
# df_cpinm(壓合recipe)-> RMS_PinlamRecipe (RecipeName)

query_cpinm = "SELECT macno as 'MacNo',in_code as 'MacRecipe' , * FROM cpinm WHERE credate >= '2024-01-01' AND NOT (credate >= '2025-01-01' AND credate <= '2025-05-26') "

df_cpinm = pd.read_sql(query_cpinm, conn5)
df_cpinm = strip_all_columns(df_cpinm)

C:\Users\wilson_liu\AppData\Local\Temp\ipykernel_9360\2726373186.py:6: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_cpinm = pd.read_sql(query_cpinm, conn5)


In [17]:
df_cpinm.columns

Index(['MacNo', 'MacRecipe', 'cpinnbr', 'cpin_pro', 'work_ctr', 'credate',
       'shift', 'lot_nbr', 'part_nbr', 'ver', 'layer', 'macno', 'ssample',
       'itemno', 'mtl_desc', 'cplot_nbr', 'qty', 'iopen', 'in_type', 'in_qty',
       'kg', 'in_code', 'b_desc', 'in_time', 'memo', 'hdi', 'fail_qty',
       'in_open', 'userid', 'lastdate', 'cdate', 'facekg', 'Outsourcing',
       'Bonding', 'StackPN', 'ChkRecNbr', 'CarrierNo'],
      dtype='object')

In [18]:
# 'prod_nbr', 'layer_full' 若相同，留最新
# 確保 creat_date 是 datetime 格式
df_cpinm['creat_date'] = pd.to_datetime(df_cpinm['credate'])

# 找出每個 group 裡 Credate 最大的那一筆資料的 index
idx = df_cpinm.groupby(['part_nbr', 'lot_nbr', 'layer'])['credate'].idxmax()

# 選出這些資料
df_cpinm = df_cpinm.loc[idx].reset_index(drop=True)

In [19]:
df_cpinm = df_cpinm[['MacNo', 'MacRecipe', 'lot_nbr', 'layer', 'part_nbr']]
df_cpinm = df_cpinm.drop_duplicates()

<p style="font-size: 16px; color: blue;">RC、廠商和基板型號</p>

In [20]:
# 以下三個串 RC

query_zms2 = "SELECT * FROM zms2"
df_zms2 = pd.read_sql(query_zms2, conn2)
df_zms2 = strip_all_columns(df_zms2)

query_zbom2 = "SELECT * FROM zbom2"
df_zbom2 = pd.read_sql(query_zbom2, conn2)
df_zbom2 = strip_all_columns(df_zbom2)

query_bsitemm = "SELECT * FROM bsitemm"
df_bsitemm = pd.read_sql(query_bsitemm, conn3)
df_bsitemm = strip_all_columns(df_bsitemm)

C:\Users\wilson_liu\AppData\Local\Temp\ipykernel_9360\2876753893.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_zms2 = pd.read_sql(query_zms2, conn2)
C:\Users\wilson_liu\AppData\Local\Temp\ipykernel_9360\2876753893.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_zbom2 = pd.read_sql(query_zbom2, conn2)
C:\Users\wilson_liu\AppData\Local\Temp\ipykernel_9360\2876753893.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_bsitemm = pd.read_sql(query_bsitemm, conn3)


In [21]:
query_RC = '''
WITH step0 AS (
    SELECT * 
    FROM (
        SELECT *, RANK() OVER (PARTITION BY prod_nbr, mtl_type ORDER BY ver DESC) AS rk 
        FROM zbom2
    ) t
    WHERE rk = 1
),step1 as(
	SELECT 
		a.*, 
		COALESCE(b1.itemno_1, b2.itemno_1) AS itemno_1  -- 先對非基板，對完再對基板的部分
	FROM zms2 a
	LEFT JOIN step0 b1 
		ON a.prod_nbr = b1.prod_nbr 
	   AND a.layer = b1.layer 
	   AND a.mtl_type = b1.mtl_type 
	   AND a.item_nbr = b1.mtl_types
	LEFT JOIN step0 b2 
		ON a.prod_nbr = b2.prod_nbr 
	   AND a.layer_2 = b2.layer 
	   AND a.mtl_type = b2.mtl_type 
)
select a.*, b.itemcnm from step1 a
left join erp..bsitemm b on a.itemno_1 = b.itemno
'''

In [22]:
# df_zms2 = df_zms2.drop(columns=['mtl_desc3', 'grade', 'mtl_flag', 'use_qty', 'mtl_desc4', 'mtl_desc5', 'layer_2ThickTole']) # 去掉 mtl_desc3、grade，都是' '
# df_zms2 = df_zms2.sort_values(by=['prod_nbr', 'layer', 'item_nbr', 'layer_2']).reset_index(drop=True)
df_RC = pd.read_sql(query_RC, conn2)
df_RC = strip_all_columns(df_RC)

C:\Users\wilson_liu\AppData\Local\Temp\ipykernel_9360\3902887923.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_RC = pd.read_sql(query_RC, conn2)


<p style="font-size: 14px; color: green;"> RC value</p>

In [23]:
# df_RC[df_RC['prod_nbr'] == '366S938BA8C']

In [24]:
# 建立新欄位 'RC_value' 的data: df_RC
df_RC['RC_value'] = df_RC['itemcnm'].str.extract(r'RC[:\s]?(\d+)', expand=False)
df_RC['RC_value'] = pd.to_numeric(df_RC['RC_value'], errors='coerce')

In [25]:
df_RC['RC_value'].notna().sum() # 237476

np.int64(237476)

<p style="font-size: 12px; color: red;"> df_RC 整理成 1 to 1</p>

In [26]:
x = df_RC[['prod_nbr', 'layer', 'item_nbr', 'mtl_type', 'mtl_desc1','mtl_desc2', 'vend_nbr', 'layer_2', 'layer_2Thick', 'RC_value']]
x1 = split_layer_rows(x)

In [27]:
# 用df_DM篩選df_RC，為 df_RC_sub

# 1. 把 df_merge 的key value組合取出來，轉成唯一列表
key_pairs = (
    df_DM[['Part_nbr', 'Layer']]
    .drop_duplicates()
    .rename(columns={'Part_nbr': 'prod_nbr', 'Layer': 'layer'})   # 改名方便後面 inner merge
)

# 2. 直接 inner merge，保留「兩邊共有」的 (prod_nbr, layer)
x1_sub = (
    x1
      .merge(key_pairs, on=['prod_nbr', 'layer'], how='inner')
      .reset_index(drop=True)
)

<p style="font-size: 12px; color: red;"> check料號: 120SO07DA8C、999S790AB8G (這沒有RC)、107S002C48N、395A024A33H(沒銅箔)</p>

In [28]:
# 預處理：保留原始索引
df = x1_sub.sort_values(by=['prod_nbr', 'layer', 'item_nbr']).reset_index()

# 初始化所有 plate_ 欄位
cols_to_plate = [col for col in df.columns 
                if col not in ['prod_nbr', 'layer', 'item_nbr', 'mtl_type', 'index']]
for col in cols_to_plate:
    df[f'plate_{col}'] = None

drop_indexes = []
result_df = df.copy()

# 處理每個 group
for (prod, layer), group in df.groupby(['prod_nbr', 'layer']):
    group = group.sort_values('item_nbr').reset_index(drop=True)
    
    # 找出所有非膠片的索引位置（基板或銅箔）
    non_adhesive_indices = group[group['mtl_type'].isin(['基板', '銅箔'])].index.tolist()
    
    # 處理每兩個非膠片之間的膠片
    for i in range(len(non_adhesive_indices)-1):
        start_idx = non_adhesive_indices[i]
        end_idx = non_adhesive_indices[i+1]
        
        # 取得區間內的膠片
        adhesives = group.loc[start_idx+1:end_idx-1]
        adhesives = adhesives[adhesives['mtl_type'] == '膠片'].sort_values('item_nbr')
        
        # 如果有膠片才處理
        if len(adhesives) > 0:
            prev_row = group.loc[start_idx]
            next_row = group.loc[end_idx]

            prev_idx = result_df[result_df['index'] == prev_row['index']].index[0]
            next_idx = result_df[result_df['index'] == next_row['index']].index[0]

            # 特殊情況：連續相同 item_nbr 的基板
            if (prev_row['mtl_type'] == '基板' and next_row['mtl_type'] == '基板' and
                prev_row['item_nbr'] == next_row['item_nbr']):
                
                # 將膠片按順序分配給連續基板
                for k, (adhesive_idx, adhesive_row) in enumerate(adhesives.iterrows()):
                    target_pos = start_idx + k
                    if target_pos <= end_idx:
                        target_row = group.loc[target_pos]
                        target_df_idx = result_df[result_df['index'] == target_row['index']].index[0]
                        for col in cols_to_plate:
                            result_df.loc[target_df_idx, f'plate_{col}'] = adhesive_row[col]
                        drop_indexes.append(adhesive_row['index'])
            else:
                if len(adhesives) == 1:
                    # 單一膠片，複製兩次
                    first_adhesive = adhesives.iloc[0]
                    second_adhesive = adhesives.iloc[0]
                elif len(adhesives) >= 2:
                    # 保留第一與最後一片膠片
                    first_adhesive = adhesives.iloc[0]
                    second_adhesive = adhesives.iloc[-1]
                    for j in range(1, len(adhesives)-1):
                        drop_indexes.append(adhesives.iloc[j]['index'])

                # 分配第一膠片給較近的那邊
                first_dist_to_prev = abs(int(first_adhesive['item_nbr']) - int(prev_row['item_nbr']))
                first_dist_to_next = abs(int(first_adhesive['item_nbr']) - int(next_row['item_nbr']))
                first_target_idx = prev_idx if first_dist_to_prev <= first_dist_to_next else next_idx
                for col in cols_to_plate:
                    result_df.loc[first_target_idx, f'plate_{col}'] = first_adhesive[col]
                drop_indexes.append(first_adhesive['index'])

                # 分配第二膠片給另一邊
                second_dist_to_prev = abs(int(second_adhesive['item_nbr']) - int(prev_row['item_nbr']))
                second_dist_to_next = abs(int(second_adhesive['item_nbr']) - int(next_row['item_nbr']))
                second_target_idx = next_idx if second_dist_to_next <= second_dist_to_prev else prev_idx
                for col in cols_to_plate:
                    result_df.loc[second_target_idx, f'plate_{col}'] = second_adhesive[col]
                drop_indexes.append(second_adhesive['index'])

# 刪除所有膠片
x2 = result_df[~result_df['index'].isin(drop_indexes)].reset_index(drop=True)
x2 = x2[x2['mtl_type'] != '膠片']

# 可選：刪除完全為空的 plate_ 欄位
x2 = x2.dropna(axis=1, how='all')
x2 = x2.drop(columns=['index', 'plate_layer_2'])

In [29]:
# x2 = x2.drop(columns=['index'])
x2 = x2.rename(columns={'prod_nbr':'Part_nbr',
                        'layer':'Layer',
                        'layer_2':'d_Layer'})

<p style="font_size:12; color:red;">隨機抽取料號測試</p>

In [37]:
# random_nbr = x2['Part_nbr'].unique()
# random_nbr = random.choice(x2['Part_nbr'].unique())
# random_nbr.upper()

In [38]:
# x2[x2['Part_nbr'] == '353S097B57F']

<p style="font-size: 14px; color: green;"> 合併df_DM 和 x2(攤平膠片訊息與RC值)</p>

In [30]:
df_merge1 = df_DM.merge(
    x2,
    left_on=['Part_nbr', 'Layer', 'd_Layer'],
    right_on=['Part_nbr', 'Layer', 'd_Layer'],
    how='inner'
)

<p style="font-size: 14px; color: green;"> df_merge1 和 df_cpinm</p>

In [31]:
df_cpinm = df_cpinm.rename(columns={'part_nbr':'Part_nbr',
                              'layer':'Layer',
                              'lot_nbr':'Lot_nbr'})

df_merge2 = df_merge1.merge(
    df_cpinm,
    left_on=['Part_nbr', 'Lot_nbr', 'Layer'],
    right_on=['Part_nbr', 'Lot_nbr', 'Layer'],
    how='inner'
)

<p style="font-size: 14px; color: green;"> df_merge2 和 df_residual_flat(殘銅率)</p>

In [32]:
df_residual_flat = df_residual_flat.rename(columns={'prod_nbr':'Part_nbr','d_Layer':'d_Layer'})

df_merge3 = df_merge2.merge(
     df_residual_flat,
    left_on=['Part_nbr', 'd_Layer'],
    right_on=['Part_nbr', 'd_Layer'],
    how='inner'
)

In [33]:
x = df_merge3#[df_merge3['Part_nbr'] == '305E268C9BC'] 

<p style="font-size: 16px; color: blue;">內層帳縮</p>

<p style = "font_size: 16; color: blue;"> 合併df_total and 內層漲縮值解析結果</p>

In [34]:
df_VAR = pd.read_excel('內層漲縮值解析結果.xlsx')
df_VAR = df_VAR.rename(columns={'Panel_ID':'Part_nbr'}) 

In [35]:
x = x.copy()
x['Part_nbr'] = x['Part_nbr'].astype(str).str.strip()
x['d_Layer'] = x['d_Layer'].astype(str).str.strip()
df_VAR['Panel_ID'] = df_VAR['Part_nbr'].astype(str).str.strip()

# 先把 df2 設為以 Panel_ID 為 index，方便查欄位
df_VAR_indexed = df_VAR.set_index("Part_nbr")

# 套用
x[['short', 'long']] = x.apply(get_layer_scaling, axis=1)

In [36]:
# 用df_VAR篩選x

# 1. 把 df_VAR 的key value組合取出來，轉成唯一列表
key_pairs = (
    df_VAR[['Part_nbr']]
    .drop_duplicates())   # 改名方便後面 inner merge

# 2. 直接 inner merge，保留「兩邊共有」的 (Part_nbr)
x = (
    x.merge(key_pairs, on=['Part_nbr'], how='inner')
      .reset_index(drop=True)
)

In [37]:
a = x#[x['Part_nbr']=='187S051A88C']
# a

In [38]:
a = a.drop(columns=['item_nbr', 'mtl_type'])

<p style = "font_size: 16; color: blue;">轉換欄位 mtl_desc1: parse_mtl_desc</p>

In [39]:
# 轉換欄位 mtl_desc1: parse_mtl_desc
parsed_df = a['mtl_desc1'].apply(parse_mtl_desc)
parsed_df = parsed_df.rename(columns={'mtl_desc2':'substrate_thickness'})
a_parsed = pd.concat([a, parsed_df], axis=1)
# a_parsed

In [40]:
a_parsed = a_parsed[['Part_nbr', 'Lot_nbr', 'Layer', 'd_Layer', 'CamX', 'CamY', 'AVGX', 'AVGY', 'Credate', 'MacNo', 'MacRecipe',
                     'mtl_desc1', 'mtl_desc2', 'vend_nbr', 'layer_2Thick', 'plate_mtl_desc1','plate_mtl_desc2', 'plate_vend_nbr', 'plate_layer_2Thick',
                     'plate_RC_value','Residual Copper Rate','glass_fabric_thickness(mm)', 'use_cu_upper_thickness',
                     'use_cu_lower_thickness', 'use_cu_type', 'glass_fabric_type', 'parse_failed_desc', 'short', 'long', 'd_PSVarX', 'd_PSVarY']]

In [41]:
a_parsed.shape

(57189, 31)

<p style = "font_size: 16; color: blue;">測試 a_parsed</p>

In [42]:
a_parsed = a_parsed.copy()
a_parsed.replace(' ', np.nan, inplace=True)

for col in a_parsed.columns:
    nan_count = a_parsed[col].isna().sum()
    print(f"There are {nan_count} Nan in {col}.")

There are 0 Nan in Part_nbr.
There are 0 Nan in Lot_nbr.
There are 0 Nan in Layer.
There are 0 Nan in d_Layer.
There are 0 Nan in CamX.
There are 0 Nan in CamY.
There are 0 Nan in AVGX.
There are 0 Nan in AVGY.
There are 0 Nan in Credate.
There are 0 Nan in MacNo.
There are 0 Nan in MacRecipe.
There are 0 Nan in mtl_desc1.
There are 0 Nan in mtl_desc2.
There are 0 Nan in vend_nbr.
There are 0 Nan in layer_2Thick.
There are 0 Nan in plate_mtl_desc1.
There are 0 Nan in plate_mtl_desc2.
There are 0 Nan in plate_vend_nbr.
There are 0 Nan in plate_layer_2Thick.
There are 5965 Nan in plate_RC_value.
There are 0 Nan in Residual Copper Rate.
There are 786 Nan in glass_fabric_thickness(mm).
There are 786 Nan in use_cu_upper_thickness.
There are 786 Nan in use_cu_lower_thickness.
There are 1417 Nan in use_cu_type.
There are 1819 Nan in glass_fabric_type.
There are 56403 Nan in parse_failed_desc.
There are 3653 Nan in short.
There are 3653 Nan in long.
There are 0 Nan in d_PSVarX.
There are 0 Nan

In [46]:
# a_parsed.to_excel('df_total.xlsx', index=False) # , na_rep='NA'

In [60]:
# a_parsed.columns

In [43]:
# 檢查完全重複的行
duplicates = a_parsed.duplicated(subset=['Part_nbr', 'Lot_nbr', 'Layer', 'd_Layer'], keep=False)
duplicated_rows = a_parsed[duplicates].sort_values(['Part_nbr', 'Lot_nbr'])

if duplicated_rows.empty:
    print("✅ 無完全重複的組合")
else:
    print(f"⚠️ 發現 {len(duplicated_rows)} 筆重複記錄：")
    # display(duplicated_rows.head(20))

✅ 無完全重複的組合


In [44]:
# 檢查是否唯一
dupes = a_parsed[a_parsed.duplicated(subset=['Part_nbr', 'Lot_nbr', 'Layer', 'd_Layer'], keep=False)]
if not dupes.empty:
    print("❗ 有重複的組合：")
    display(dupes)
else:
    print("✅ 所有 (Part_nbr, Lot_nbr, Layer, d_Layer) 組合唯一")


✅ 所有 (Part_nbr, Lot_nbr, Layer, d_Layer) 組合唯一


In [63]:
# e = a_parsed.drop(columns=['MacNo','MacRecipe'])
# # e['Credate'] = pd.to_datetime(e['Credate']).dt.date
# e = e.drop_duplicates()
# e.shape

In [64]:
# # 檢查完全重複的行
# duplicates = e.duplicated(subset=['Part_nbr', 'Lot_nbr', 'Layer', 'd_Layer'], keep=False)
# duplicated_rows = e[duplicates].sort_values(['Part_nbr', 'Lot_nbr'])
# # duplicated_rows = duplicated_rows[duplicated_rows['Part_nbr']=='187S051A88C']

# if duplicated_rows.empty:
#     print("✅ 無完全重複的組合")
# else:
#     print(f"⚠️ 發現 {len(duplicated_rows)} 筆重複記錄：")
#     # display(duplicated_rows.head(40))
#     # duplicated_rows.to_excel('duplicated_rows3.xlsx', index=True)